This file is meant to complete the data cleaning and create two datasets ready for analysis: one for outfield players, and one for keepers.

In [1]:
# Libraries
import pandas as pd
from unidecode import unidecode

In [2]:
## Importing datasets
standard = pd.read_csv('C:/Users/Toby Chiu/Desktop/Coding/Personal Projects/Personal/wins-above-rep-soccer/data/raw-df/big5_player_standard.csv')
shooting = pd.read_csv('C:/Users/Toby Chiu/Desktop/Coding/Personal Projects/Personal/wins-above-rep-soccer/data/raw-df/big5_player_shooting.csv')
passing = pd.read_csv('C:/Users/Toby Chiu/Desktop/Coding/Personal Projects/Personal/wins-above-rep-soccer/data/raw-df/big5_player_passing.csv')
passing_types = pd.read_csv('C:/Users/Toby Chiu/Desktop/Coding/Personal Projects/Personal/wins-above-rep-soccer/data/raw-df/big5_player_passing_types.csv')
gca = pd.read_csv('C:/Users/Toby Chiu/Desktop/Coding/Personal Projects/Personal/wins-above-rep-soccer/data/raw-df/big5_player_gca.csv')
defense = pd.read_csv('C:/Users/Toby Chiu/Desktop/Coding/Personal Projects/Personal/wins-above-rep-soccer/data/raw-df/big5_player_defense.csv')
possession = pd.read_csv('C:/Users/Toby Chiu/Desktop/Coding/Personal Projects/Personal/wins-above-rep-soccer/data/raw-df/big5_player_possession.csv')
playing_time = pd.read_csv('C:/Users/Toby Chiu/Desktop/Coding/Personal Projects/Personal/wins-above-rep-soccer/data/raw-df/big5_player_playing_time.csv')
misc = pd.read_csv('C:/Users/Toby Chiu/Desktop/Coding/Personal Projects/Personal/wins-above-rep-soccer/data/raw-df/big5_player_misc.csv')
keepers_base = pd.read_csv('C:/Users/Toby Chiu/Desktop/Coding/Personal Projects/Personal/wins-above-rep-soccer/data/raw-df/big5_player_keepers.csv')
keepers_adv = pd.read_csv('C:/Users/Toby Chiu/Desktop/Coding/Personal Projects/Personal/wins-above-rep-soccer/data/raw-df/big5_player_keepers_adv.csv')
outfield_valuation = pd.read_csv('C:/Users/Toby Chiu/Desktop/Coding/Personal Projects/Personal/wins-above-rep-soccer/data/raw-df/outfield_player_valuations.csv')
keepers_valuation = pd.read_csv('C:/Users/Toby Chiu/Desktop/Coding/Personal Projects/Personal/wins-above-rep-soccer/data/raw-df/keeper_player_valuations.csv')


## Data Cleaning

Manual Inspection reveals a low % rate of error in consistency across datasets.

We can conduct some data cleaning to reduce systematic errors when joining datasets.

We will use a full outer join to combine our datasets using the "unique_ID" as the link, remedying any errors later.

In [3]:
outfield = [standard, shooting, passing, passing_types, gca, defense, possession, playing_time, misc, outfield_valuation]
keepers = [keepers_base, keepers_adv, keepers_valuation]

In [4]:
# Creating a unique ID per club - in case player decides to transfer mid-season
# This approach can standardize club names across FBref and TM datasets

# Creating a squad name "translater"
clubs_fbref = shooting["Squad"].copy()
clubs_tm = outfield_valuation["squad"].copy()
club_translate = pd.concat([clubs_fbref, clubs_tm], axis=1, join='inner')

for col in ["squad", "Squad"]:
    club_translate[col] = club_translate[col].apply(unidecode)

# club_translate.to_csv("C:/Users/Toby Chiu/Desktop/Coding/Personal Projects/Personal/wins-above-rep-soccer/data/club_translate.csv")

In [5]:
# Using club_translater to change all club names from TM datasets to FBref club names

club_translate =  pd.read_csv("C:/Users/Toby Chiu/Desktop/Coding/Personal Projects/Personal/wins-above-rep-soccer/data/club_translate.csv")
translater = dict(zip(club_translate.TM_squad, club_translate.FBref_Squad))

outfield_valuation = outfield_valuation.replace({"squad": translater})
keepers_valuation = keepers_valuation.replace({"squad": translater})

# update unique_ID with club - new format for unique_ID will be name_year_club
for df in outfield[:-1]:
    df["unique_ID"] = df[["unique_ID", "Squad"]].agg("_".join, axis=1)
for df in keepers[:-1]:
    df["unique_ID"] = df[["unique_ID", "Squad"]].agg("_".join, axis=1)


outfield_valuation["unique_ID"] = outfield_valuation[["unique_ID", "squad"]].agg("_".join, axis=1)
keepers_valuation["unique_ID"] = keepers_valuation[["unique_ID", "squad"]].agg("_".join, axis=1)


In [6]:
# Removing excess columns
outfield_valuation = outfield_valuation[outfield_valuation.columns[~outfield_valuation.columns.isin(["comp_name", "region", "country", "season_start_year", "squad", "player_name"])]]
keepers_valuation = keepers_valuation[keepers_valuation.columns[~keepers_valuation.columns.isin(["comp_name", "region", "country", "season_start_year", "squad", "player_name"])]]

outfield = [standard, shooting, passing, passing_types, gca, defense, possession, playing_time, misc, outfield_valuation]
keepers = [keepers_base, keepers_adv, keepers_valuation]

In [7]:
# Data Cleaning: Transliterating names with accents into standard English across all datasets
# UPDATE: works now
for df in outfield:
    temp = df["unique_ID"]
    temp = temp.apply(unidecode)
    df["unique_ID"] = temp

for df in keepers:
    temp = df["unique_ID"]
    temp = temp.apply(unidecode)
    df["unique_ID"] = temp

In [8]:
# Full outer join on both datasets

merged_df = outfield[0].copy()


for i in range(1, 10):
    merged_df = pd.merge(merged_df, outfield[i], how='outer', on='unique_ID', suffixes=('', '_remove'))
    merged_df.drop([x for x in merged_df.columns if 'remove' in x], axis = 1, inplace=True)
    merged_df = merged_df[merged_df.columns[~merged_df.columns.isin(["Unnamed: 0"])]]

outfield_final = merged_df.drop_duplicates()

merged_keepers_df = keepers[0].copy()

for i in range(1, 3):
    merged_keepers_df = pd.merge(merged_keepers_df, keepers[i], how='outer', on='unique_ID', suffixes=('', '_remove'))
    merged_keepers_df.drop([x for x in merged_keepers_df.columns if 'remove' in x], axis = 1, inplace=True)
    merged_keepers_df = merged_keepers_df[merged_keepers_df.columns[~merged_keepers_df.columns.isin(["Unnamed: 0"])]]

keepers_final = merged_keepers_df.drop_duplicates()

In [ ]:
# Sorting datasets
outfield_final = outfield_final.sort_values("unique_ID")
keepers_final = keepers_final.sort_values("unique_ID")

outfield_final.reset_index(drop=True)
keepers_final.reset_index(drop=True)

In [9]:
outfield_final.to_csv("C:/Users/Toby Chiu/Desktop/Coding/Personal Projects/Personal/wins-above-rep-soccer/data/outfield_no_edit.csv")
keepers_final.to_csv("C:/Users/Toby Chiu/Desktop/Coding/Personal Projects/Personal/wins-above-rep-soccer/data/keepers_no_edit.csv")

We will make final changes through excel:

- If a player transfers, his stats are tied to his old club, but his value is tied to his new club.

  In this case, we can **formulate a predicted valuation**.

    1. Take the player's value from the same year at a different club
    2. Take the player's last year valuation

- Lots of players have empty columns - remove players with no stats 
